# Initial Python model

In [ ]:
import torch

x = torch.randn(5, 784) # 5 samples, images of 28*28 = 784 pixels
weights = torch.randn(784, 10)
bias = torch.zeros(10)

print(weights.shape)
print(bias.shape)

## Matrix multiplication

### Method a)


In [ ]:
def matmul(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br, "Incompatible shapes for matrix multiplication"
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i,j] += a[i,k] * b[k,j]
    return c



In [ ]:
%time t1 = matmul(x, weights)

'''
%%time is a magic command. It's a part of IPython.
%%time prints the wall time for the entire cell whereas %time gives you the time for first line only
'''

### Method b)
Element-wise operations. Operators like (+, -, *, /, >, <, ==) are usually element-wise

In [ ]:
def matmul_b(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br, "Incompatible shapes for matrix multiplication"
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i,j] = (a[i,:] * b[:,j]).sum() # Python calls C code to do this
    return c

In [ ]:
%time t1 = matmul(x, weights)

%time t1 = matmul_b(x, weights)

%timeit -n 10  _ = matmul_b(x, weights)



In [ ]:
#export 
from exp.nb_00 import *

def near(a, b): return torch.allclose(a, b, rtol=1e-5, atol=1e-8)
def test_near(a,b): test(a,b, near)



In [ ]:
# If it runs without an assertion error, it means the two functions are equivalent
test_near(matmul(x, weights), matmul_b(x, weights))


### c) Matmul with broadcasting

In [ ]:
def matmul_c(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br, "Incompatible shapes for matrix multiplication"
    c = torch.zeros(ar, bc)
    for i in range(ar):
        # c[i,:] = (a[i,:] * b).sum(0) # Python calls C code to do this
        # c[i] = (a[i].unsqueeze(-1) * b).sum(dim=0) # Python calls C code to do this
        c[i] = (a[i][...,None] * b).sum(dim=0) # Python calls C code to do this

    return c



In [ ]:
print(x.shape)
print(x[None, 0].shape)
print(x[0, None].shape)
print(x[0].shape)
print(x[0][...,None].shape)
print()

print(x[0].unsqueeze(-1).shape) 
print(weights.shape)


In [ ]:

%timeit -n 10  _ = matmul(x, weights)
%timeit -n 10  _ = matmul_b(x, weights)
%timeit -n 10  _ = matmul_c(x, weights)



### d) Matmul with Einsum

In [ ]:
def matmul_d(a,b):
    return torch.einsum('ij,jk->ik', a, b)

In [ ]:
print("Time with index matmul:")
%timeit -n 10  _ = matmul(x, weights)
print('Time with ":"" row and col selection:')
%timeit -n 10  _ = matmul_b(x, weights)
print("Time with broadcasting")
%timeit -n 10  _ = matmul_c(x, weights)
print("Time with einsum:")
%timeit -n 10  _ = matmul_d(x, weights)



### e) Matmul from Pytorch 
Use of BLAS (Basic Linear Algebra Subprograms)

In [ ]:
print("Time with index matmul:")
%timeit -n 10  _ = matmul(x, weights)
print('Time with ":"" row and col selection:')
%timeit -n 10  _ = matmul_b(x, weights)
print("Time with broadcasting")
%timeit -n 10  _ = matmul_c(x, weights)
print("Time with einsum:")
%timeit -n 10  _ = matmul_d(x, weights)
print("Time Pytorch Matmul:")
%timeit -n 10  t2 = x.matmul(weights)





# Tests

In [ ]:
t1 = matmul(x, weights)
t2 = x.matmul(weights)

test_near(t1, t2)


# Programming languages Jeremy mentions:

+ APL
+ J
+ K
+ Halide

- Polyhedral compillation

+ Tensor comprehension